# Detectron2 Model Evaluation 

This notebook includes portions that builds D2 in the SageMaker Studio kernel and performs evaluation with both D2's `COCOEvaluator` and cartucho's mAP library. 

It will need to run on a PyTorch GPU Optimized instance. 

In [ ]:
%%bash 

# Environment setup for building D2 locally in a SageMaker Studio instance 

export FORCE_CUDA="1"
export TORCH_CUDA_ARCH_LIST="Volta"

pip install torchvision torch --upgrade 
pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI' 'git+https://github.com/facebookresearch/fvcore' 'git+https://github.com/facebookresearch/detectron2.git' google-colab scikit-image sagemaker-inference

## Register datasets 

This will register the COCO-formatted dataset into Detectron2 so that it can parse the metadata. 

**NOTE: Sometimes it doesn't parse the classes... Note the code snippet:** 

```python 
MetadataCatalog.get(val_dataset_name).thing_classes = ['OBJECT_1', 'OBJECT_2', 'OBJECT_3', 'OBJECT_4', 'OBJECT_5']
MetadataCatalog.get(val_dataset_name).thing_dataset_id_to_contiguous_id={1: 0, 2: 1, 3: 2, 4: 3, 5: 4}
```

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data.catalog import MetadataCatalog, DatasetCatalog
import os
import json

# VALIDATION SPLIT 
val_dataset_name = "val"
val_dataset_location = "../data/"
val_annotation_file = "val.json"
val_image_dir = "val"

register_coco_instances(val_dataset_name, {}, os.path.join(val_dataset_location, val_annotation_file), 
                        os.path.join(val_dataset_location, val_image_dir))

val_meta = MetadataCatalog.get(val_dataset_name)

# TRAIN SPLIT 
train_dataset_name = "train"
train_dataset_location = "../data/"
train_annotation_file = "train.json"
train_image_dir = "train"

register_coco_instances(train_dataset_name, {}, os.path.join(train_dataset_location, train_annotation_file), 
                        os.path.join(train_dataset_location, train_image_dir))

train_meta = MetadataCatalog.get(train_dataset_name)

# TEST SPLIT 
test_dataset_name = "test"
test_dataset_location = "../data/"
test_annotation_file = "test.json"
test_image_dir = "test"

register_coco_instances(test_dataset_name, {}, os.path.join(test_dataset_location, test_annotation_file), 
                        os.path.join(test_dataset_location, test_image_dir))

test_meta = MetadataCatalog.get(test_dataset_name)

### Specify class maps in case D2 didn't parse it well

In [ ]:
# register_coco_instances seems to fail in registering the classes -- append here 

MetadataCatalog.get(val_dataset_name).thing_classes = ['OBJECT_1', 'OBJECT_2', 'OBJECT_3', 'OBJECT_4', 'OBJECT_5'] # put more here! 
MetadataCatalog.get(val_dataset_name).thing_dataset_id_to_contiguous_id={1: 0, 2: 1, 3: 2, 4: 3, 5: 4} # Update the mapping here as so too for class 0 to be background!

In [ ]:
# Do initial configuration
from detectron2.config import get_cfg
from detectron2 import model_zoo

config_file = "source/faster_rcnn_R_101_FPN_3x.yaml"
# config_file = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(config_file))
cfg.DATASETS.TRAIN = (train_dataset_name,)
cfg.DATASETS.TEST = (val_dataset_name,)  
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.OUTPUT_DIR = "models_dir"

In [ ]:
# Local Inference 

import cv2
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow

cfg.MODEL.WEIGHTS = "models_dir/model.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05   # set the output confidence threshold 
# cfg.DATASETS.TEST = (val_dataset_name, )
predictor = DefaultPredictor(cfg)

test_pics = '<PATH_TO_LOCAL_TEST_IMGS>'

for img_ in os.listdir(test_pics): 
    img_name = test_pics + img_ 

    im = cv2.imread(img_name)
    outputs = predictor(im)

    v = Visualizer(im[:, :, ::-1],
                   metadata=cb_val_meta, 
                   scale=0.8)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# D2 Evaluation 

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultTrainer

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)

evaluator = COCOEvaluator(test_dataset_name, cfg, False, output_dir="./eval_output/")
val_loader = build_detection_test_loader(cfg, test_dataset_name)
inference_on_dataset(trainer.model, val_loader, evaluator)

## Cartucho mAP setup

The following code sets up cartucho mAP calculations. It will be dumping following their guidelines of creating `.txt` files in both the `ground-truth` and `detection-results` directories in the `mAP/input/` directory. Each file within those directories will correspond to the image, with each line representing an object in the image. 

[cartucho github repo](https://github.com/Cartucho/mAP)

In [ ]:
# Use Cartucho's Object Detection Evaluation Repo 
! git clone https://github.com/Cartucho/mAP.git

# Clear out the dirs that the evaluation metrics are calculated on 
! rm mAP/input/ground-truth/*
! rm mAP/input/detection-results/*

# Grab the test set manifest from S3 that was uploaded when you ran `dataprep.ipynb` (or locally if you have it)
! aws s3 cp <S3_PATH_TO_TEST.MANIFEST> <LOCAL_DIR_PATH> 

### `detection-results` generation 

In [ ]:
! pip install tqdm
from tqdm import tqdm

classID_name = {
    0: 'OBJECT1',
    1: 'OBJECT2',
    2: 'OBJECT3',
    3: 'OBJECT4',
    4: 'OBJECT5',
}

detection_results_dir = '/d2/mAP/input/detection-results/'

cfg.MODEL.WEIGHTS = "d2/models_dir/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05   # set to 0.05 to get all detections but not get the random ones 
predictor = DefaultPredictor(cfg)

test_pics = '<LOCAL_PATH_TO_DIRECTORY_OF_TEST_SET_IMGS>'

results = [] # (fname, outputs)

for img_ in os.listdir(test_pics): 
    try: 
        img_name = test_pics + img_ 
    #     img_name = test_pics + 'ponytail_su_00151.jpg' # earphones

        im = cv2.imread(img_name)
        outputs = predictor(im)

        results.append( (img_, outputs) )
    except:
        print(img_name)
    
    # Uncomment this section if you want to visualize 
#     v = Visualizer(im[:, :, ::-1],
#                    metadata=cb_val_meta, 
#                    scale=0.8)
#     out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#     cv2_imshow(out.get_image()[:, :, ::-1])


for fname, outputs in tqdm(results):
    preds = outputs["instances"].to("cpu")
    boxes = preds.pred_boxes.tensor.numpy()
    scores = preds.scores.tolist()
    classes = preds.pred_classes.tolist()
    
    with open(f'{detection_results_dir}{fname[:-4]}.txt', 'w') as f:
        for i in range(len(boxes)):
            left, top, right, bot = boxes[i] #x0, y0, x1, y1
            f.write(f'{classID_name[classes[i]]} {scores[i]} {int(left)} {int(top)} {int(right)} {int(bot)}\n') # <class_name> <confidence> <left> <top> <right> <bottom>

### `ground-truth` generation

In [ ]:
! pip install jsonlines
import jsonlines
import json

groundtruth_dir = '/d2/mAP/input/ground-truth/'
detection_results_dir = 'input/detection-results/'

job_name = '20200818-relabel-filter-fresh'

with jsonlines.open('20200818-relabel-filter-fresh-test.manifest', 'r') as reader:
    for desc in tqdm(reader):
        fname = desc['source-ref'].split('/')[-1][:-4]
        with open(f'{groundtruth_dir}{fname}.txt', 'w') as f:
            for obj in desc[job_name]['annotations']: 
                f.write(f'{classID_name[obj["class_id"]]} {obj["left"]} {obj["top"]} {obj["left"] + obj["width"]} {obj["top"] + obj["height"]}\n')


## Run the cartucho mAP generation! 

The following cell will execute mAP's `main.py`, it will dump visualizations into the `mAP/output/` directory. 

In [ ]:
! python mAP/main.py 